In [17]:
import os
import cv2
import numpy as np
from pathlib import Path
import copy
import time

In [ ]:
# Define the path to the input and output folders
input_dir = 'Path to the original dataset'
output_dir = "Path to  save the anonimyzed dataset"

In [19]:
# define the anonimyzation parametes
b= 16
eps = 0.1
m = 16

In [20]:
# Create the output directory if it does not exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [21]:
def create_dp_pixelated_image(x, b, m, eps):
    img = torch.squeeze(x)
    img1 = np.transpose(img.cpu().detach().numpy(), (1, 2, 0))
    
    
    pix = Pix(img1, b)
    DP_Pixed = DP_Pix(pix, b, m, eps)
    
    DP_Pixed1 = torch.from_numpy(np.transpose(DP_Pixed, (2, 0, 1)))
    DP_Pixed_tensor = torch.unsqueeze(DP_Pixed1, 0)
    
    return DP_Pixed_tensor

In [22]:
def Pix(X_arr, b):
    X_noise = []
    
    (b_channel, g_channel, r_channel) = cv2.split(X_arr)
    new_b = copy.copy(b_channel)
    new_g = copy.copy(g_channel)
    new_r = copy.copy(r_channel)
    x, y = new_b.shape
    
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_b[i:i + b, j:j + b] = new_b[i:i + b, j:j + b].mean(axis=(0, 1))
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_g[i:i + b, j:j + b] = new_g[i:i + b, j:j + b].mean(axis=(0, 1))
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_r[i:i + b, j:j + b] = new_r[i:i + b, j:j + b].mean(axis=(0, 1))
                
    X_noise = cv2.merge((new_b,new_g,new_r))
    
    return X_noise

In [23]:
def DP_Pix(X_arr, b, m, eps):
    
    
    X_noise = []
    (b_channel, g_channel, r_channel) = cv2.split(X_arr)
    new_b = copy.copy(b_channel)
    new_g = copy.copy(g_channel)
    new_r = copy.copy(r_channel)
    x, y = new_b.shape
    
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_b[i:i + b, j:j + b] = new_b[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                       + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_g[i:i + b, j:j + b] = new_g[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                       + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_r[i:i + b, j:j + b] = new_r[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                           + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
                
    X_noise = cv2.merge((new_b,new_g,new_r))
    return X_noise

In [8]:
start_time = time.time()
counter = 0
# Loop through each folder in the input directory
for foldername in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, foldername)
    if os.path.isdir(folder_path):
        # Create the corresponding subfolder in the output directory
        output_folder_path = os.path.join(output_dir, foldername)
        Path(output_folder_path).mkdir(parents=True, exist_ok=True)

        # Loop through each file in the folder
        for filename in os.listdir(folder_path):
            counter+=1
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Skipping {file_path}")
                    continue

                img = Pix(img, b)
                img = DP_Pix(img, b, m, eps)

                # Save the processed image to the output folder
                output_file_path = output_folder_path +'/' + str(counter)+'.jpg'
                cv2.imwrite(output_file_path, img)
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")


Overall execution time: 974.3671021461487 seconds
All images processed.
